In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision import transforms
import torch.optim as optim
from tqdm.auto import tqdm

In [ ]:
# Подключаем диск
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Функция загрузки данных
def load_data_from_path(path):
    data = []
    labels = []
    print(f"Загрузка данных из: {path}")
    for filename in tqdm(sorted(os.listdir(path))):
        try:
            # Собираем полный путь к файлу
            full_path = os.path.join(path, filename)
            # Открываем, конвертируем в RGB и меняем размер
            image = Image.open(full_path).convert('RGB').resize((180, 180))
            data.append(np.array(image))

            # Определяем метку: 0 для 'cat', 1 для 'dog'
            if 'cat' in filename:
                labels.append(0)
            else:
                labels.append(1)
        except Exception as e:
            print(f"Не удалось обработать файл {filename}: {e}")

    return np.array(data), np.array(labels)

In [ ]:
# Запуск загрузки и визуализация

base_path = "/content/drive/MyDrive/my_colab_data/Dogs-Cats/"
train_path = os.path.join(base_path, "Train/")
test_path = os.path.join(base_path, "Test/")

# Загружаем данные
train_dataset_np, y_train_np = load_data_from_path(train_path)
test_dataset_np, y_test_np = load_data_from_path(test_path)

print(f"\nЗагружено {len(train_dataset_np)} обучающих изображений.")
print(f"Загружено {len(test_dataset_np)} тестовых изображений.")

In [ ]:
# Визуализация для проверки
print("\nПримеры изображений (кошки):")
plt.figure(figsize=(10, 5))
for i in range(4):
    plt.subplot(2, 4, i + 1)
    plt.imshow(train_dataset_np[i])
    plt.title("Cat")
    plt.axis('off')

print("\nПримеры изображений (собаки):")
dog_indices = np.where(y_train_np == 1)[0]
for i in range(4):
    plt.subplot(2, 4, i + 5)
    plt.imshow(train_dataset_np[dog_indices[i]])
    plt.title("Dog")
    plt.axis('off')
plt.show()

In [ ]:
# Предобработка данных и определение модели

# Нормализуем данные в диапазон [0, 1] и меняем порядок осей: (N, H, W, C) -> (N, C, H, W)
train_tensor = torch.tensor(train_dataset_np / 255.0, dtype=torch.float32).permute(0, 3, 1, 2)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1)

test_tensor = torch.tensor(test_dataset_np / 255.0, dtype=torch.float32).permute(0, 3, 1, 2)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1)

# Создаем датасеты PyTorch
train_tensor_dataset = TensorDataset(train_tensor, y_train_tensor)
test_tensor_dataset = TensorDataset(test_tensor, y_test_tensor)

# Определяем устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

# Определяем архитектуру модели
class CNNBinaryClassifier(nn.Module):
    def __init__(self):
        super(CNNBinaryClassifier, self).__init__()
        # Блок 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        # Блок 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        # Блок 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2) # Пул после каждого блока

        # Динамическое вычисление размера для FC слоя
        with torch.no_grad():
            # Прогоняем пустышку через сверточную часть, чтобы узнать размер выхода
            dummy_input = torch.zeros(1, 3, 180, 180)
            x = self.pool(F.relu(self.bn1(self.conv1(dummy_input))))
            x = self.pool(F.relu(self.bn2(self.conv2(x))))
            x = self.pool(F.relu(self.bn3(self.conv3(x))))
            self.flattened_size = x.view(1, -1).size(1)

        # Полносвязная часть
        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 1) # Один выход для бинарной классификации

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1) # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # Без сигмоиды, так как BCEWithLogitsLoss
        return x

Используемое устройство: cuda


In [ ]:
# Обучение модели БЕЗ аугментации

# Гиперпараметры
learning_rate = 0.0001
num_epochs = 30
batch_size = 32

# Создаем загрузчики данных
train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size)

# Инициализируем модель, лосс и оптимизатор
model_no_aug = CNNBinaryClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_no_aug.parameters(), lr=learning_rate)

# Списки для хранения метрик
history_no_aug = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

print("Начинаем обучение модели БЕЗ аугментации")

for epoch in range(num_epochs):
    # Обучение
    model_no_aug.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc=f"Эпоха {epoch+1}/{num_epochs} [Обучение]"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_no_aug(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.sigmoid(outputs) > 0.5
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    epoch_train_loss = running_loss / total_train
    epoch_train_acc = correct_train / total_train
    history_no_aug['train_loss'].append(epoch_train_loss)
    history_no_aug['train_acc'].append(epoch_train_acc)

    # Валидация
    model_no_aug.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_no_aug(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            preds = torch.sigmoid(outputs) > 0.5
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    epoch_val_loss = val_loss / total_val
    epoch_val_acc = correct_val / total_val
    history_no_aug['val_loss'].append(epoch_val_loss)
    history_no_aug['val_acc'].append(epoch_val_acc)

    print(f"Эпоха {epoch+1}/{num_epochs} | "
          f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

print("\nОбучение без аугментации завершено")

In [ ]:
# Визуализация результатов
def plot_history(history, title):
    plt.figure(figsize=(14, 5))

    # График точности
    plt.subplot(1, 2, 1)
    plt.plot(history['train_acc'], label='Точность на обучении')
    plt.plot(history['val_acc'], label='Точность на валидации')
    plt.title('Точность модели')
    plt.ylabel('Точность')
    plt.xlabel('Эпоха')
    plt.legend()

    # График потерь
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Потери на обучении')
    plt.plot(history['val_loss'], label='Потери на валидации')
    plt.title('Потери модели')
    plt.ylabel('Потери')
    plt.xlabel('Эпоха')
    plt.legend()

    plt.suptitle(title, fontsize=16)
    plt.show()

plot_history(history_no_aug, "Результаты обучения БЕЗ аугментации")

In [ ]:
# Определение аугментаций

# Трансформации для обучающего набора данных
# ToPILImage() нужен для numpy массивов
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5), # Случайное горизонтальное отражение
    transforms.RandomRotation(degrees=20),    # Случайный поворот на угол до 20 градусов
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Случайное изменение яркости/контраста
    transforms.ToTensor(), # Преобразуем в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Стандартная нормализация
])

# Трансформации для тестового набора
test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Визуализируем, как работает аугментация
print("Примеры аугментации для одного изображения:")
image_np = train_dataset_np[0] # Берем первую кошку
plt.figure(figsize=(12, 12))
for i in range(9):
    augmented_image = train_transforms(image_np)
    # Делаем обратное преобразование для корректного отображения
    img_to_show = augmented_image.permute(1, 2, 0).numpy()
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_to_show = std * img_to_show + mean
    img_to_show = np.clip(img_to_show, 0, 1)

    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img_to_show)
    plt.axis("off")
plt.show()

In [ ]:
# CustomDataset и модель с Dropout

class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

# Создаем датасеты с аугментацией
train_dataset_aug = CustomImageDataset(train_dataset_np, y_train_np, transform=train_transforms)
test_dataset_aug = CustomImageDataset(test_dataset_np, y_test_np, transform=test_transforms)

# Модель, идентичная предыдущей, но с добавлением Dropout
class CNNWithDropout(nn.Module):
    def __init__(self):
        super(CNNWithDropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)

        # Динамический расчет размера
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 180, 180)
            x = self.pool(F.relu(self.bn1(self.conv1(dummy_input))))
            x = self.pool(F.relu(self.bn2(self.conv2(x))))
            x = self.pool(F.relu(self.bn3(self.conv3(x))))
            self.flattened_size = x.view(1, -1).size(1)

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.dropout = nn.Dropout(p=0.5) # Добавляем Dropout с вероятностью 0.5
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x) # Применяем Dropout перед последним слоем
        x = self.fc2(x)
        return x

In [ ]:
# Обучение модели c аугментацией

# Гиперпараметры
learning_rate = 0.0001
num_epochs = 80
batch_size = 32

# Создаем загрузчики данных
train_loader_aug = DataLoader(train_dataset_aug, batch_size=batch_size, shuffle=True)
test_loader_aug = DataLoader(test_dataset_aug, batch_size=batch_size)

# Инициализация
model_with_aug = CNNWithDropout().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_with_aug.parameters(), lr=learning_rate)

history_with_aug = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

print("Начинаем обучение модели С аугментацией и Dropout...")

for epoch in range(num_epochs):
    # Фаза обучения
    model_with_aug.train()
    running_loss, correct_train, total_train = 0.0, 0, 0
    for images, labels in tqdm(train_loader_aug, desc=f"Эпоха {epoch+1}/{num_epochs} [Обучение]"):
        images, labels = images.to(device), labels.to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model_with_aug(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        preds = torch.sigmoid(outputs) > 0.5
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    epoch_train_loss = running_loss / total_train
    epoch_train_acc = correct_train / total_train
    history_with_aug['train_loss'].append(epoch_train_loss)
    history_with_aug['train_acc'].append(epoch_train_acc)

    # Фаза валидации
    model_with_aug.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader_aug:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            outputs = model_with_aug(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            preds = torch.sigmoid(outputs) > 0.5
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    epoch_val_loss = val_loss / total_val
    epoch_val_acc = correct_val / total_val
    history_with_aug['val_loss'].append(epoch_val_loss)
    history_with_aug['val_acc'].append(epoch_val_acc)

    print(f"Эпоха {epoch+1}/{num_epochs} | "
          f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

print("\nОбучение с аугментацией завершено.")

# Визуализация результатов
plot_history(history_with_aug, "Результаты обучения С аугментацией и Dropout")